In [1]:
# %load_ext tensorboard
# Clear any logs from previous runs
# !rm -rf ./logs/

In [2]:
import numpy as np
import pandas as pd
import time, os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.utils import to_categorical
from tensorboard.plugins.hparams import api as hp

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

import random
from math import floor
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from scipy.integrate import odeint
import scipy
import pandas as pd
np.random.seed(0)

In [3]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)

(740, 2)

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.8
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((592, 1), (148, 1), (592, 2), (148, 2))

In [6]:
# loss -> val_loss
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min',
                                   min_delta=0.0,
                                   patience=200,
                                   restore_best_weights=True, verbose=True)
                                   ]

n_cols = 1

In [7]:
HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([2, 4, 8, 16]))
HP_LAYERS =    hp.HParam('layers', hp.Discrete([1, 2, 3, 4]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([50, 100, 150, 200]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([1e-4,1e-3]))
# HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([50, 100, 150, 200]))
# HP_NUM_UNITS4 = hp.HParam('num_units4', hp.Discrete([2, 5, 10]))
# HP_DROPOUT = hp.HParam('dropout', hp.RealInterval(0.0, 0.2))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'Adadelta']))
# HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam']))

In [8]:
epochs=200

In [9]:
# sess = tf.Session()
# writer = tf.summary.FileWriter("/tmp/tfvgg", sess.graph)
# init = tf.initialize_all_variables()
# sess.run(init)
# with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
#     hp.hparams_config(
#         hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3, HP_NUM_UNITS4,
#                  HP_OPTIMIZER, HP_BATCHSIZE],
#         metrics=[hp.Metric('loss', display_name="Loss")])

In [10]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning3').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS,
                 HP_NUM_UNITS,
                 HP_LEARNING, 
                 HP_BATCHSIZE],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [11]:
def train_test_model(hparams):    
    
    # Train LSTM model and predict on validation set
    model = tf.keras.Sequential()
    model.add(Dense(int(X_train.shape[1])))
#     model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(2, activation='linear'))
     
    optimizer = tf.keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING], beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss='mse', 
            metrics=['mean_squared_error'])
    
    # Run with 1 epoch to speed things up for demo purposes

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=hparams[HP_BATCHSIZE], shuffle=False, verbose=0)

    _, loss = model.evaluate(X_test, Y_test)
    
    return loss

In [12]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)
        return loss

In [13]:
session_num = 0
datos = []

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
            for batch_size in HP_BATCHSIZE.domain.values:
                t = time.time()
                hparams = {

                    HP_LAYERS: deep_layers,
                    HP_NUM_UNITS: num_units,
                    HP_LEARNING: learning_rate,
                    HP_BATCHSIZE: batch_size,
                }
                run_name = "run-%d" % session_num
                print('\n--- Starting trial: %s' % run_name)
                print({h.name: hparams[h] for h in hparams})
                score = run('logs/hparam_tuning3/' + run_name, hparams)
                t = time.time()-t
                session_num += 1
                print("Loss:", score, "Tiempo transcurrido:", t)
            
            datos.append([deep_layers, num_units, learning_rate, batch_size, score, t])

print(session_num)


--- Starting trial: run-0
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0721 - mean_squared_error: 0.0721
Loss: 0.07212551683187485 Tiempo transcurrido: 78.68402576446533

--- Starting trial: run-1
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.1107 - mean_squared_error: 0.1107
Loss: 0.11066646873950958 Tiempo transcurrido: 41.7295126914978

--- Starting trial: run-2
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 1ms/step - loss: 0.4998 - mean_squared_error: 0.4998
Loss: 0.49984678626060486 Tiempo transcurrido: 24.287182331085205

--- Starting trial: run-3
{'layers': 1, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.6527 - mean_squared_error: 0.6527
Loss: 0.652683436870575 

5/5 [==============================] - 0s 2ms/step - loss: 0.0362 - mean_squared_error: 0.0362
Loss: 0.0361664779484272 Tiempo transcurrido: 16.691831350326538

--- Starting trial: run-32
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 3ms/step - loss: 0.0326 - mean_squared_error: 0.0326
Loss: 0.03261382505297661 Tiempo transcurrido: 141.90365052223206

--- Starting trial: run-33
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0371 - mean_squared_error: 0.0371
Loss: 0.037094373255968094 Tiempo transcurrido: 83.53283858299255

--- Starting trial: run-34
{'layers': 2, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0945 - mean_squared_error: 0.0945
Loss: 0.09448590874671936 Tiempo transcurrido: 41.56967639923096

--- Starting trial: run-35
{'layers': 2, 'num_units': 

5/5 [==============================] - 0s 2ms/step - loss: 0.0318 - mean_squared_error: 0.0318
Loss: 0.03175904601812363 Tiempo transcurrido: 26.17475914955139

--- Starting trial: run-63
{'layers': 2, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 1ms/step - loss: 0.0493 - mean_squared_error: 0.0493
Loss: 0.04931722581386566 Tiempo transcurrido: 15.873072862625122

--- Starting trial: run-64
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0334 - mean_squared_error: 0.0334
Loss: 0.033423006534576416 Tiempo transcurrido: 83.48709511756897

--- Starting trial: run-65
{'layers': 3, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 4}
5/5 [==============================] - 0s 2ms/step - loss: 0.0401 - mean_squared_error: 0.0401
Loss: 0.04008065536618233 Tiempo transcurrido: 45.15489077568054

--- Starting trial: run-66
{'layers': 3, 'num_units':

5/5 [==============================] - 0s 2ms/step - loss: 0.0422 - mean_squared_error: 0.0422
Loss: 0.04217454046010971 Tiempo transcurrido: 82.62004542350769

--- Starting trial: run-94
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0327 - mean_squared_error: 0.0327
Loss: 0.032703738659620285 Tiempo transcurrido: 35.593634843826294

--- Starting trial: run-95
{'layers': 3, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.0343 - mean_squared_error: 0.0343
Loss: 0.03430866822600365 Tiempo transcurrido: 19.639758825302124

--- Starting trial: run-96
{'layers': 4, 'num_units': 50, 'learning_rate': 0.0001, 'batch_size': 2}
5/5 [==============================] - 0s 2ms/step - loss: 0.0297 - mean_squared_error: 0.0297
Loss: 0.029650216922163963 Tiempo transcurrido: 93.45386362075806

--- Starting trial: run-97
{'layers': 4, 'num_units

5/5 [==============================] - 0s 2ms/step - loss: 0.0795 - mean_squared_error: 0.0795
Loss: 0.07947611063718796 Tiempo transcurrido: 146.09634566307068

--- Starting trial: run-125
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 4}
5/5 [==============================] - 0s 3ms/step - loss: 0.0421 - mean_squared_error: 0.0421
Loss: 0.04212126135826111 Tiempo transcurrido: 76.25984477996826

--- Starting trial: run-126
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 8}
5/5 [==============================] - 0s 2ms/step - loss: 0.0385 - mean_squared_error: 0.0385
Loss: 0.038466647267341614 Tiempo transcurrido: 43.13245177268982

--- Starting trial: run-127
{'layers': 4, 'num_units': 200, 'learning_rate': 0.001, 'batch_size': 16}
5/5 [==============================] - 0s 2ms/step - loss: 0.0488 - mean_squared_error: 0.0488
Loss: 0.04880204051733017 Tiempo transcurrido: 27.075721502304077
128


In [14]:
filename = "historial_jla_tunning.txt"
df = pd.DataFrame(datos, columns = ["Deep size", "Num units", "Learning rate", "Batch size", "MSE", "Tiempo de ejecución"])

df.sort_values(by=["MSE", "Tiempo de ejecución"], ascending=[True, True], ignore_index=True, inplace=True)

df.to_csv(filename, header=True, index=False, sep='\t', mode='w') # a=append, w=overwrite

In [15]:
df.head(2)

,Deep size,Num units,Learning rate,Batch size,MSE,Tiempo de ejecución
0,2,50,0.0010,16,0.031442,23.044686
1,4,150,0.0001,16,0.032232,21.347682


In [16]:
total_time = float(np.sum(df[["Tiempo de ejecución"]])/60)

print("Elapsed time {:.3f} minutes".format(total_time))

Elapsed time 10.446 minutes


In [17]:
df["Tiempo de ejecución"]

0     23.044686
1     21.347682
2     19.639759
3     16.691831
4     16.341290
5     19.596324
6     17.295284
7     24.289318
8     16.182329
9     22.788187
10    18.297002
11    23.044182
12    14.458308
13    21.077509
14    16.815729
15    16.615918
16    16.948540
17    20.853782
18    15.754957
19    27.075722
20    15.873073
21    22.195918
22    22.874772
23    21.180001
24    16.620122
25    17.795614
26    31.925913
27    18.706803
28    21.453212
29    16.587430
30    15.540130
31    17.854442
Name: Tiempo de ejecución, dtype: float64